In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1

class Language:
    def __init__(self, name):
        self.name = name
        self.word_to_index = {}
        self.word_to_count = {}
        self.index_to_word = {0: "SOS", 1: "EOS"}
        self.num_words = 2

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word_to_index:
            self.word_to_index[word] = self.num_words
            self.word_to_count[word] = 1
            self.index_to_word[self.num_words] = word
            self.num_words += 1
        else:
            self.word_to_count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")

    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    sentence_pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]

    if reverse:
        sentence_pairs = [list(reversed(p)) for p in sentence_pairs]
        input_language = Language(lang2)
        output_language = Language(lang1)
    else:
        input_language = Language(lang1)
        output_language = Language(lang2)

    return input_language, output_language, sentence_pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# Only use pairs where the data has the prefix above
def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [6]:
def prepare_data(lang1, lang2, reverse=False):
    input_language, output_language, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_language.add_sentence(pair[0])
        output_language.add_sentence(pair[1])
    print("Counted words:")
    print(input_language.name, input_language.num_words)
    print(output_language.name, output_language.num_words)
    return input_language, output_language, pairs


In [7]:
input_language, output_language, pairs = prepare_data('eng', 'fra', True)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je fais attention a ne pas trop depenser .', 'i m careful not to spend too much .']


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden_cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden_cell = self.lstm(output, hidden_cell)
        return output, hidden_cell

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden_cell):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden_cell = self.lstm(output, hidden_cell)
        output = self.softmax(self.out(output[0]))
        return output, hidden_cell

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [10]:
# Split a sentence by ' ' and return a list of the tokens for each word
def indexes_from_sentence(language, sentence):
    return [language.word_to_index[word] for word in sentence.split(' ')]

# Call the above on a sentence, After calling, add the EOS_token (int id) to the gotten list
def tensor_from_sentence(language, sentence):
    indexes = indexes_from_sentence(language, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

#Return a tuple of 2 tensors, one input_tensor and another an output_tensor
def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_language, pair[0])
    target_tensor = tensor_from_sentence(output_language, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5

In [12]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden()
    encoder_cell = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    input_tensor = input_tensor.flip(0)

    for it in range(input_length):
        encoder_output, (encoder_hidden, encoder_cell) = encoder(input_tensor[it], (encoder_hidden, encoder_cell))

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    target_length_used = 0

    if use_teacher_forcing:
        target_length_used = target_length

        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            loss += criterion(decoder_output, target_tensor[jt])
            decoder_input = target_tensor[jt]

    else:
        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[jt])

            target_length_used += 1

            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length_used

In [13]:
# Helper functions used to time the optimizations
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [14]:
def train_iterations(
    encoder,
    decoder,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # Get n_iters training pairs
    # In this example, we are effectively doing SGD with batch size 1
    training_pairs = [
        tensors_from_pair(random.choice(pairs))
        for i in range(n_iters)
    ]
    
    # The loss; either NLLLoss if you use log sigmoids or CrossEntropyLoss if you use logits
    criterion = nn.NLLLoss()

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # Train on the input, target pair
        loss = train(
            input_tensor, target_tensor, encoder,
            decoder, encoder_optimizer, decoder_optimizer, criterion
        )
        
        # Update the total loss and the plot loss
        # We can plot and print at different granularities
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, it / n_iters),
                                         it, it / n_iters * 100, print_loss_avg))

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [25]:
# For a certain input, get the predicted output sentence
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_language, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()
        encoder_cell = encoder.init_hidden()
        
        for it in range(input_length):
            encoder_output, (encoder_hidden, encoder_cell) = encoder(
                input_tensor[it],
                (encoder_hidden, encoder_cell)
            )
            
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for jt in range(max_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input,
                (decoder_hidden, decoder_cell)
            )
            topv, topi = decoder_output.data.topk(1)
            
            decoded_words.append(output_language.index_to_word[topi.item()])
            
            if topi.item() == EOS_token:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [16]:
hidden_size = 256
cell_size = 256
encoder = EncoderRNN(input_language.num_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_language.num_words).to(device)

In [17]:
train_iterations(encoder, decoder, 75000, print_every=5000)

2m 56s (- 41m 8s) (5000 6%) 3.1025
5m 47s (- 37m 38s) (10000 13%) 2.4799
8m 48s (- 35m 12s) (15000 20%) 2.1971
11m 49s (- 32m 31s) (20000 26%) 1.9727
14m 48s (- 29m 36s) (25000 33%) 1.7689
17m 39s (- 26m 29s) (30000 40%) 1.6329
20m 30s (- 23m 26s) (35000 46%) 1.4579
23m 9s (- 20m 15s) (40000 53%) 1.3710
26m 11s (- 17m 27s) (45000 60%) 1.2365
29m 11s (- 14m 35s) (50000 66%) 1.1035
32m 9s (- 11m 41s) (55000 73%) 0.9741
35m 4s (- 8m 46s) (60000 80%) 0.9405
38m 6s (- 5m 51s) (65000 86%) 0.8476
41m 12s (- 2m 56s) (70000 93%) 0.7682
44m 7s (- 0m 0s) (75000 100%) 0.6981


In [26]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_randomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        pair = random.choice(pairs)
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])

        if '<EOS>' == output_words[-1]:
            output_words.pop()

        output_sentence = ' '.join(output_words)
        score = 100.0 * sentence_bleu([pair[1].split(' ')], output_words, weights=[0.5, 0.5])
        bleu_scores.append(score)
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    print('The mean BLEU score is: ', np.mean(bleu_scores))

In [27]:
evaluate_randomly(encoder, decoder)

/Users/frankchao/opt/anaconda3/envs/ML/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


The mean BLEU score is:  13.09091670636161
